## **<font color="red">State</font>**
- **Bypassing State Injection with `InstructionProvider`:**
  - In some cases, you might want to use `{{ and }}` literally in your instructions without triggering the state injection mechanism. For example, you might be writing instructions for an agent that helps with a templating language that uses the same syntax.
  - To achieve this, you can provide a function to the `instruction` parameter instead of a string. This function is called an `InstructionProvider`. When you use an InstructionProvider, the ADK will not attempt to inject state, and your instruction string will be passed to the model as-is.
  - The `InstructionProvider` function receives a ReadonlyContext object, which you can use to access session state or other contextual information if you need to build the instruction dynamically.

In [1]:
import os
from config import config
from google.adk.agents import LlmAgent
from google.adk.agents.readonly_context import ReadonlyContext

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

# This is an InstrucitonProvider
def custom_instruction_provider(context: ReadonlyContext)-> str:
    # You can optionally use the context to build the instruction
    # For this example, we'll return a static string with literal braces.
    return "This is an instruction with {{literal_braces}} that will not be replaced."\
    

story_generator = LlmAgent(
    name="StoryGenerator",
    model="gemini-2.5-flash",
    instruction=custom_instruction_provider
)




- If you want to both use an `InstructionProvider` and inject state into your instructions, you can use the `inject_session_state` utility function.
- **Benefits of Direct Injection:**
  - ***Clarity:** Makes it explicit which parts of the instruction are dynamic and based on session state.
  - **Reliability:** Avoids relying on the LLM to correctly interpret natural language instructions to access state.
  - **Maintainability:** Simplifies instruction strings and reduces the risk of errors when updating state variable names.
- **Relation to Other State Access Methods**
  - This direct injection method is specific to LlmAgent instructions. Refers to the following seciton for more information on other state access methods.

In [2]:
import os
from config import config
from google.adk.agents import LlmAgent
from google.adk.agents.readonly_context import ReadonlyContext
from google.adk.utils import instructions_utils

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

# Dynamic Context Provider
async def my_dynamic_instruction_provider(context: ReadonlyContext) -> str:
    template = "This is a {adjective} instruction with {{literal_braces}}."
    # This will inject the 'adjective' state variable but leave the literal braces.
    return await instructions_utils.inject_session_state(template, context)

story_generator = LlmAgent(
    name="StoryGenerator",
    model="gemini-2.5-flash",
    instruction=my_dynamic_instruction_provider
)




- **The Right Way to Modify State:**
  - When you need to change the session state, the correct and safest method is to *directly modify the `state` object on the** `Context` provided to your function (e.g., `callback_context.state['my_key'] = 'new_value'`). This is considered "direct state manipulation" in the right way, as the framework automatically tracks these changes.
  - This is critically different from directly modifying the `state` on a `Session` object you retrieve from the `SessionService` (e.g., `my_session.state['my_key'] = 'new_value'`). *You should avoid this*, as it bypasses the ADK's event tracking and can lead to lost data. The "Warning" section at the end of this page has more details on this important distinction.
- State should always be updated as part of adding an Event to the session history using `session_service.append_event()`. This ensures changes are tracked, persistence works correctly, and updates are thread-safe.

#### **<font color='blue'>1. The Easy Way: `output_key` (for Agent Text Responses)</font>**
- This is the simplest method for saving and agent's final text response directly into the state. When defining your `LlmAgent`, specify me `output_key`:

In [3]:
import os
from config import config
from google.adk.agents import LlmAgent
from google.adk.sessions import InMemorySessionService, Session
from google.adk.runners import Runner
from google.genai.types import Content, Part

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

# Agent
greeting_agent = LlmAgent(
    name="Greeter",
    model="gemini-2.5-flash",
    instruction="Generate a short, friendly greeting",
    output_key="last_greeting" # Save response to state['last_greeting']
)

# --- Setup Runner and Session ---
APP_NAME, USER_ID, SESSION_ID = "state_app", "user1", "session1"

# Session: Memory
session_service = InMemorySessionService()

# Runner: AgentExecutor
runner = Runner(
    agent=greeting_agent,
    app_name=APP_NAME,
    session_service=session_service
)

# Create Session: Memory Initializaiton
session = await session_service.create_session(app_name=APP_NAME, 
                                               user_id=USER_ID,
                                               session_id=SESSION_ID)
print(f"Initial state: {session.state}")

# --- Run the Agent ---
# Runner handles calling append_event, which uses the output_key to automatically create the state_delta.
user_message = Content(parts=[Part(text="Hello")])  # PromptTemplate

for event in runner.run(user_id=USER_ID,
                        session_id=SESSION_ID,
                        new_message=user_message):
    if event.is_final_response():
        print(f"Agent Response: ", event.content)
print('\n')

# --- Check Updated State ---
updated_session = await session_service.get_session(app_name=APP_NAME, 
                                                    user_id=USER_ID,
                                                    session_id=SESSION_ID)
print(f"State after agent run: {updated_session.state}")


Initial state: {}
Agent Response:  parts=[Part(
  text='Hi there!'
)] role='model'


State after agent run: {'last_greeting': 'Hi there!'}


In [4]:
# --- Run the Agent ---
# Runner handles calling append_event, which uses the output_key to automatically create the state_delta.
user_message = Content(parts=[Part(text="What is thermodynamic?")])  # PromptTemplate

for event in runner.run(user_id=USER_ID,
                        session_id=SESSION_ID,
                        new_message=user_message):
    if event.is_final_response():
        print(f"Agent Response: ", event.content)
print('\n')

# --- Check Updated State ---
updated_session = await session_service.get_session(app_name=APP_NAME, 
                                                    user_id=USER_ID,
                                                    session_id=SESSION_ID)
print(f"State after agent run: {updated_session.state}")


Agent Response:  parts=[Part(
  text="""Thermodynamics is a branch of physics that deals with **heat, work, and temperature, and their relation to energy, radiation, and the physical properties of matter.**

In simpler terms, it's the study of how energy is transferred and transformed in different systems, especially focusing on how heat can be converted into other forms of energy (like work) and vice versa. It's governed by a set of fundamental laws that describe these processes."""
)] role='model'


State after agent run: {'last_greeting': "Thermodynamics is a branch of physics that deals with **heat, work, and temperature, and their relation to energy, radiation, and the physical properties of matter.**\n\nIn simpler terms, it's the study of how energy is transferred and transformed in different systems, especially focusing on how heat can be converted into other forms of energy (like work) and vice versa. It's governed by a set of fundamental laws that describe these processes."}


- Behind the scenes, the `Runner` uses the `output_key` to create the necessary `EventActions` with a `state_delta` and calls `append_event`.

#### **<font color='blue'>2. Then Standard Way: `EventActions.state_delta` (for Complex Updates)</font>**
- For more complex scenarios (updating multiple keys, non-string values, specific scopes like `user:` or `app:`, or updates not tried directly to the agent's final text), you manually construct the `state_delta` within `EventActions` .

#### **What each Import Does**
##### **InMemorySessionService**
1. Manages sessions in RAM
2. Used for **testing** / **development**.
3. Data is lost when program stops.
##### **Session**
1. Represents a conversation session
2. Contains:
   - metadata
   - state
   - event history
##### **Event**
1. Represents a single action in the session.
2. Could be:
   - user message
   - agent response
   - tool call
   - system update
##### **EventActions**
1. Defines what should happen when event is appended.
2. Most important here:
   - `state_delta` :-> update session state

In [5]:
from google.adk.sessions import InMemorySessionService, Session
from google.adk.events import Event, EventActions
from google.genai.types import Part, Content
import time

# --- Setup ---
session_service = InMemorySessionService()

APP_NAME, USER_ID, SESSION_ID = "state_app_manual", "user2", "session2"

# Create Session: Memory Initializaiton
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
    state={"user:login_count": 0, "task_status": "idle"}
)
print(f"Initial state: {session.state}")

# --- Define State Changes ---
current_time = time.time()
state_changes = {
    "task_status": "active",  # Update session state
    "user:login_count": session.state.get("user:login_count", 0) + 1, # Update user state
    "user:last_login_ts": current_time, # Add user state
    "temp:validation_needed": True, # Add temporary state (will be discarded)
}

# --- Create Event with Actions ---
actions_with_update = EventActions(state_delta=state_changes)
# This event might represent an internal system action, not just an agent response
system_event = Event(
    invocation_id= "inv_login_update",
    author="system", # Or 'agent', 'tool' etc.
    actions=actions_with_update,
    timestamp=current_time
    # content might be None or represent the action taken
)

# --- Append the Event (This updates the state) ---
await session_service.append_event(session, system_event)
print("`append_event` called with explicit state delta.")

# --- Check Updated State ---
updated_session = await session_service.get_session(app_name=APP_NAME,
                                                    user_id=USER_ID,
                                                    session_id=SESSION_ID
                                                   )
print(f"State after event: {updated_session.state}")


Initial state: {'task_status': 'idle', 'user:login_count': 0}
`append_event` called with explicit state delta.
State after event: {'task_status': 'active', 'user:login_count': 1, 'user:last_login_ts': 1771166956.7865512}


In [6]:
# Check the Memory State
import asyncio
import time
from google.adk.sessions import InMemorySessionService
from google.adk.events import Event, EventActions


APP_NAME, USER_ID, SESSION_ID = "state_app_manual", "user2", "session2"

# ------------------------------------
# 1️⃣ Create Session
# ------------------------------------
session_service = InMemorySessionService()

session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
    state={"user:login_count": 0}
)

print("Initial State:", session.state)
print("-" * 50)

# ------------------------------------
# 2️⃣ Define Conversation Function
# ------------------------------------
async def conversation():

    current_time = time.time()

    # Always fetch latest session
    latest_session = await session_service.get_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )

    current_count = latest_session.state.get("user:login_count", 0)

    state_changes = {
        "user:login_count": current_count + 1,
        "user:last_login_ts": current_time
    }

    event = Event(
        invocation_id=f"login_event_{current_time}",
        author="system",
        actions=EventActions(state_delta=state_changes),
        timestamp=current_time
    )

    await session_service.append_event(latest_session, event)

    updated_session = await session_service.get_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )

    print("After Login:", updated_session.state)

# ------------------------------------
# 3️⃣ 5 Times Login
# ------------------------------------
await conversation()
await conversation()
await conversation()
await conversation()
await conversation()

print("-" * 50)

# ------------------------------------
# 4️⃣ Final Check
# ------------------------------------
final_session = await session_service.get_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)

print("Final login_count:",
      final_session.state.get("user:login_count"))



Initial State: {'user:login_count': 0}
--------------------------------------------------
After Login: {'user:login_count': 1, 'user:last_login_ts': 1771166956.7971616}
After Login: {'user:login_count': 2, 'user:last_login_ts': 1771166956.7971616}
After Login: {'user:login_count': 3, 'user:last_login_ts': 1771166956.7971616}
After Login: {'user:login_count': 4, 'user:last_login_ts': 1771166956.7971616}
After Login: {'user:login_count': 5, 'user:last_login_ts': 1771166956.7971616}
--------------------------------------------------
Final login_count: 5


#### **<font color='blue'>3. Via `CallbackContext` or `ToolContext` (Recommended for Callbacks and Tools)</font>**
- Modifying state withing agent callbacks (e.g., `on_before_agent_call`, `on_after_agent_call`) or tool functions is best done using the `state` attribute of the `CallbackContext` or `ToolContent` provided to your function.
  - `callback_context.state['my_key'] = my_value`
  - `tool_context.state['my_key'] = my_value`
- These context objects are specifically designed to manage state changes within their respective execution scopes. When you modify `context.state`, the ADK framework ensures that these changes are automatically captured and correctly routed into the `EventActions.state_delta` for the event being generated by the callback or tool.
- This delta is then processed by the `SessionService` when the event is appended, ensuring proper persistence and tracking.
- This method abstracts away the manual creation of `EventActions` and `state_delta` for most common state update scenarios within callbacks and tools, making your code cleaner and less error-prone.


In [7]:
# In an agent callback or tool function
from google.adk.agents import callback_context # or ToolContext

def my_callback_or_tool_funciton(context: callback_context, # or ToolContext
                                 # ... Other parameters ...
                                ):
    # Update existing state
    count = context.state.get("user_action_count", 0)
    context.state["user_action_count"] = count + 1

    # Add new state
    context.state["temp: last_operation_status"] = "success"

    # State Changes are automatically part of the event's state_delta
    # ... reset of callback/tool logic


In [8]:
import os
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from config import config

os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

APP_NAME = "story_app"
USER_ID = "user_1"
SESSION_ID = "session_1"

story_generator = LlmAgent(
    name="StoryGenerator",
    model="gemini-2.5-flash",
    instruction="Write a short story about a cat."
)



session_service = InMemorySessionService()

await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
)

runner = Runner(
    agent=story_generator,
    session_service=session_service,
    app_name=APP_NAME,
)

event_generator = runner.run(
    user_id=USER_ID,
    session_id=SESSION_ID,
    new_message=types.UserContent(
        parts=[types.Part(text="Write about friendship")]
    ),
)

for event in event_generator:
    if event.content and event.content.parts:
        print(event.content.parts[0].text)


Jasper was a creature of habit and quiet dignity. His days unfolded like a well-worn scroll: sunbath on the warm patio, a meticulous grooming session, a leisurely patrol of his garden kingdom, followed by a nap beneath the rose bush. He was a sleek, charcoal-grey cat with eyes the colour of aged amber, and he preferred his solitude. It wasn't loneliness; it was simply… efficient.

Then, Pip arrived.

Pip was a whirlwind of scruffy ginger fur, all sharp angles and boundless, clumsy energy. He appeared one afternoon, a tiny, half-starved kitten who tumbled rather than walked into Jasper's carefully curated patch of sunlight. Jasper, mid-doze, opened one golden eye. Pip, startled, let out a tiny, high-pitched "mew!" and then promptly tried to pounce on a fallen leaf that wasn't there.

Jasper twitched an ear. This was an unacceptable disturbance. He let out a low, rumbling growl, a sound usually reserved for the bravest of squirrels. Pip froze, wide-eyed, then, instead of fleeing, took a 